<a href="https://colab.research.google.com/github/Storm00212/JARVIS/blob/main/JARVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain --quiet


In [3]:
!pip install sentence-transformers --quiet

In [4]:
!pip install chromadb --quiet

In [5]:
!pip install pypdf --quiet

In [6]:
!pip install transformers --quiet

In [7]:
!pip install accelerate --quiet

In [8]:
!pip install  bitsandbytes --quiet

In [9]:
!pip install torch torchvision --quiet